In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [16]:
!wget -c https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-to-pytorch/helper.py


--2018-12-30 11:46:49--  https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-to-pytorch/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [17]:
#downloading dataset
!wget -c https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
!unzip -qq Cat_Dog_data.zip

--2018-12-30 11:46:54--  https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.32.98
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.32.98|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

replace Cat_Dog_data/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [18]:


import PIL
print(PIL.PILLOW_VERSION)
# this should print 5.3.0. If it doesn't, then restart your runtime:
# Menu > Runtime > Restart Runtime

5.3.0


In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets , transforms , models

data_dir = 'Cat_Dog_data'

train_transforms = transforms.Compose([transforms.RandomRotation(30),transforms.RandomResizedCrop(224),transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),transforms.CenterCrop(224),transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

train_data = datasets.ImageFolder(data_dir+'/train' , transform=train_transforms)
test_data = datasets.ImageFolder(data_dir +'/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [0]:
#here we are using resnet and so it has two parts "features" and "fc" whereas we had features and classifer
## Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#using pre-trained resnet50 from imageNet database
model = models.resnet50(pretrained=True)

## Freeze parameters so we don't backprop through them

for param in model.parameters():
    param.requires_grad = False

#creating our own classifier with name fc (as defined in resnet50)
model.fc = nn.Sequential(nn.Linear(2048, 512),nn.ReLU(),nn.Dropout(0.2),nn.Linear(512, 2),nn.LogSoftmax(dim=1))    

#using crossentropyloss
criterion = nn.NLLLoss()

#taking steps of gradient descent with lr =0.003 and only fc parameters ,Only train the classifier parameters, feature parameters are frozen

optimizer = torch.optim.SGD(model.parameters() , lr = 0.01)

model.to(device);


In [21]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [0]:
import numpy as np

In [24]:
epochs = 5

#to have a history of losses to plot the graph
train_losses , test_losses = [] , []

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf #set initial "min" to infinity
#to put the model in training mode
model.train()
for epoch in range(epochs):
  running_loss = 0
  for images , labels in trainloader:
    images, labels = images.to(device), labels.to(device)
    # clear the gradients of all optimized variables
    optimizer.zero_grad()
    #forward pass (feed forward)forward pass: compute predicted outputs by passing inputs to the model
    outputs = model(images)
    # calculate the loss
    loss = criterion(outputs,labels)
    #gradient descent ,backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer.step()
    # update running training loss
    running_loss += loss.item()
    
  # initialize lists to monitor test loss and accuracy
  test_loss = 0
  accuracy = 0
  #we no longer gradients to be calculated
  with torch.no_grad():
    model.eval() # prep model for evaluation
    for images , labels in testloader:
      images, labels = images.to(device), labels.to(device)
      logps = model(images) # forward pass: compute predicted outputs by passing inputs to the model
      test_loss += criterion(logps,labels) #calculate the loss and update the loss
      ps = torch.exp(logps)
      top_p , top_class = ps.topk(1,dim=1)
      equals = top_class == labels.view(*top_class.shape)
      accuracy += torch.mean(equals.type(torch.FloatTensor))    
   
  train_losses.append(running_loss/len(trainloader))
  test_losses.append(test_loss/len(testloader))
  print("Epoch: {}/{}.. ".format(epoch+1, epochs),"Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),"Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),"Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
  model.train()
  # save model if validation loss has decreased
  if test_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        test_loss))
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = test_loss

Epoch: 1/5..  Training Loss: 0.234..  Test Loss: 0.067..  Test Accuracy: 0.977
Validation loss decreased (inf --> 2.691600).  Saving model ...
Epoch: 2/5..  Training Loss: 0.154..  Test Loss: 0.047..  Test Accuracy: 0.984
Validation loss decreased (2.691600 --> 1.874306).  Saving model ...
Epoch: 3/5..  Training Loss: 0.149..  Test Loss: 0.043..  Test Accuracy: 0.985
Validation loss decreased (1.874306 --> 1.719844).  Saving model ...
Epoch: 4/5..  Training Loss: 0.140..  Test Loss: 0.044..  Test Accuracy: 0.985
Epoch: 5/5..  Training Loss: 0.135..  Test Loss: 0.040..  Test Accuracy: 0.986
Validation loss decreased (1.719844 --> 1.601292).  Saving model ...
